In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('RSIL', 30),
        ('RSIS', 70),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.RSI = bt.indicators.RSI_SMA(
            self.datas[0].close, period=21)
        self.RSIL = self.params.RSIL
        self.RSIS = self.params.RSIS

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.RSI[0] < self.RSIL:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.RSI > self.RSIS:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(Curr Long Threshold %2d, Curr Short Threshold %2d) Ending Value %.2f' %
                 (self.RSIL, self.RSIS, self.broker.getvalue()), doprint=True)

In [ ]:
class SPDYAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.RSIL = self.strategy.params.RSIL
        self.RSIS = self.strategy.params.RSIS
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        outp = {
            'params': (self.RSIL, self.RSIS),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return outp

## Strategy Test

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-05-01'])


#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(SPDYAnalyzer, _name='RSI')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# cerebro.plot(volume=False, iplot=False)

## In Sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    RSIL=(30,35,40),
    RSIS=(60,65,70)
    )

# Add the Data Feed to Cerebro
cerebro.adddata(data)

#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(SPDYAnalyzer, _name='RSI')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

#slippage
cerebro.broker.set_slippage_fixed(0.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

# Run over everything
results = cerebro.run(maxcpus=1)


## Retrieve Best Strategy

In [ ]:
params  = []
n_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.RSI.get_analysis()
    params.append(r['params'])
    n_trades.append(r['ntrade'])
    profit.append(r['profit'])
    
    

fig, (prof, nt) = plt.subplots(1,2, figsize=(15,5))
_=prof.hist(profit, bins=np.arange(-5000,20000,1000))
_=nt.hist(n_trades, bins=np.arange(0,500,25))

prof.set_title('Profit Distribution')
nt.set_title('# of Trades Distribution')

In [ ]:
#Find the best Strategy (i.e. Max Profit)
prof_ind = np.argmax(profit) 
best_params = params[prof_ind]
print("Best Parameters: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of Trades: ", n_trades[prof_ind])


## In Sample Optimized run

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    RSIL=best_params[0],
                    RSIS=best_params[1])


# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

#slippage
cerebro.broker.set_slippage_fixed(0.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)


#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(SPDYAnalyzer, _name='MA')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
in_samp_sharpe = results[0].analyzers.mysharpe.get_analysis()
in_samp_drawdown = results[0].analyzers.drawdown.get_analysis()

In [ ]:
cerebro.plot(plotabove=True, iplot=True, volume=False)

## Out of Sample Optimized Run

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    RSIL=best_params[0],
                    RSIS=best_params[1])


data = bt.feeds.PandasData(dataname = df['2018-05-01':'2019-01-31'])


# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

#slippage
cerebro.broker.set_slippage_fixed(0.01, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)


#Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(SPDYAnalyzer, _name='MA')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results2 = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
out_samp_sharpe = results2[0].analyzers.mysharpe.get_analysis()
out_samp_drawdown = results2[0].analyzers.drawdown.get_analysis()

In [ ]:
cerebro.plot(plotabove=True, iplot=True, volume=False)

In [ ]:
print('in sample:')
print(in_samp_sharpe)
#print(in_samp_drawdown)
print('\n out sample:')
print(out_samp_sharpe)
#print(out_samp_drawdown)